#Going Modular: Script Mode

**What is script mode?**

Script mode uses Jupyter Notebook cell magic (special commands) to turn specific cells into Python scripts.

For example if you run the following code in a cell, you'll create a Python file called `hello_world.py`:
```
%%writefile hello_world.py
print("hello world, machine learning is fun!")
```
You could then run this Python file on the command line with:
```
python hello_world.py

>>> hello world, machine learning is fun!
```
The main cell magic we're interested in using is `%%writefile`.

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified filename.

**Project Filesystem Structure**

```
project_name/
├── scripts/
│   ├── get_setup.py
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
├── notebooks/
│   ├── pytorch_going_modular_cell_mode.ipynb
│   ├── pytorch_going_modular_script_mode.ipynb
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```    
Where the `models` and `data` folders are created in runtime, so they are not stored in git for this project.

The directory `scripts` contains several .py files. The purposes of these may be:

* `get_data.py` - a file to download the required data. **Should be called seperately.**
* `data_setup.py` - a file to prepare data.
* `engine.py` - a file containing various training functions.
* `model_builder.py` or model.py - a file to create a PyTorch model.
* `train.py` - a file to leverage all other files and train a target PyTorch model.
* `utils.py` - a file dedicated to helpful utility functions.

##Git and GitHub
Define access credentials and repo details.

The following environment variables (or secret keys) are assumed to be configured:
* GITHUB_USERNAME - github username
* GITHUB_USER_EMAIL - email configured for the github user
* GITHUB_TOKEN - github personal access token with relevant privileges (`repo` is enough)


In [1]:
from google.colab import userdata

github_user_email = userdata.get('GITHUB_USER_EMAIL')

In [2]:
token = userdata.get('GITHUB_TOKEN')
username = userdata.get('GITHUB_USERNAME')
repo = "mini-food-pic-classifier"

In [3]:
!git config --global user.name {username}
!git config --global user.email {github_user_email}
# !git config --global user.password ""

Get the latest code from the project repo.

Change the repo name and path as you need.

Create new repo in case you want to use a new repo.

In [4]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'mini-food-pic-classifier'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 90 (delta 43), reused 69 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 57.60 KiB | 2.74 MiB/s, done.
Resolving deltas: 100% (43/43), done.


Change the working path to be inside the repo folder from this point and on.

In [5]:
%cd {repo}

/content/mini-food-pic-classifier


In [6]:
# project_name = "mini-food-pic-classifier"
project_path = "/content/" + repo

Currently assume the repo does not have the most updated version of this notebook, so copy it from the google drive:
* Connect to the drive
* Copy the latest notebook from the drive to its place in the local git repo

In [7]:
import os
load_notebooks_path = os.path.join(project_path, "notebooks")
if not os.path.exists(load_notebooks_path):
  os.mkdir(load_notebooks_path)

In [8]:
notebook_name = "pytorch_going_modular_script_mode.ipynb"
notebook_path = f"\"/content/drive/MyDrive/Colab Notebooks/youtube/Learn PyTorch for deep learning in a day/05_pytorch_going_modular/{notebook_name}\""

In [9]:
!cp  {notebook_path} ./notebooks/

In [14]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/pytorch_going_modular_script_mode.ipynb

nothing added to commit but untracked files present (use "git add" to track)


###Update Git

In [15]:
!git checkout -b add_script_mode_notebook

Switched to a new branch 'add_script_mode_notebook'


In [16]:
!git add notebooks/{notebook_name}
!git commit -m "add the initial script mode notebook"

[add_script_mode_notebook 19fd07d] add the initial script mode notebook
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_script_mode.ipynb


In [17]:
!git status

On branch add_script_mode_notebook
nothing to commit, working tree clean


In [18]:
!git checkout main
!git merge add_script_mode_notebook
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 513f569..19fd07d
Fast-forward
 notebooks/pytorch_going_modular_script_mode.ipynb | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_script_mode.ipynb
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 13.23 KiB | 6.61 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/guyfuchs/mini-food-pic-classifier
   513f569..19fd07d  main -> main


##0. Creating a folder for storing Python scripts

Since we're going to be creating Python scripts out of our most useful code cells, let's create a folder for storing those scripts.

We'll call the folder `scripts` and create it using Python's `os.makedirs()` method.

In [9]:
import os

os.makedirs("scripts", exist_ok=True)

## 1. Get data
We're going to start by creating `get_data.py` script to download the `pizza_steak_sushi` datast with images of pizza, steak and sushi.

Then call this script to download the data.

In [12]:
%%writefile scripts/get_data.py
import os
import zipfile

from pathlib import Path

import requests

def download_data(project_path: str,
                  data_subpath: str,
                  file_name: str,
                  source_url: str,
                  data_sesc: str):

    # Setup path to data folder
    data_path = Path(os.path.join(project_path, "data/"))
    image_path = data_path / data_subpath

    # If the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / file_name, "wb") as f:
        request = requests.get(source_url)
        print(f"Downloading {data_sesc}...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / file_name, "r") as zip_ref:
        print(f"Unzipping {data_sesc}...")
        zip_ref.extractall(image_path)

    # Remove zip file
    os.remove(data_path / file_name)

    # Setup train and testing paths
    train_dir = image_path / "train"
    test_dir = image_path / "test"

    return train_dir, test_dir

Overwriting scripts/get_data.py


In [13]:
from scripts import get_data

train_dir, test_dir = get_data.download_data(
                        project_path = project_path ,
                        data_subpath = "pizza_steak_sushi",
                        file_name = "pizza_steak_sushi.zip",
                        source_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                        data_sesc = "pizza, steak, sushi data"
                    )

Did not find /content/mini-food-pic-classifier/data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


###Update Git

In [ ]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_get_data_script

Switched to a new branch 'cell_mode_get_data'


In [ ]:
!git add notebooks/{notebook_name}
!git add scripts/get_data.py
!git add scripts/train.py
!git commit -m "add get_data.py and call it from the train script"

[cell_mode_get_data 06fef79] Get data in cell mode
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_cell_mode.ipynb


In [ ]:
!git checkout main
!git merge add_get_data_script
!git push origin main

Everything up-to-date


##2. Create Datasets and DataLoaders
Let's turn our data into PyTorch `Dataset`'s and `DataLoader`'s and find out a few useful attributes from them such as `classes` and their lengths.

Place all the functionality into a function called `create_datadownloaders()`.

In [9]:
%%writefile scripts/data_setup.py

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKERS
):
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

  # Use ImageFolder to create dataset(s)
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Get class names
  class_names = train_data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names

Overwriting scripts/data_setup.py


In [26]:
from scripts import data_setup
from torchvision import transforms

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
                                                    train_dir=train_dir,
                                                    test_dir=test_dir,
                                                    transform=data_transform,
                                                    batch_size=32)

###Update Git

In [24]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_data_setup_script

Switched to a new branch 'add_data_setup_script'


In [25]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git add scripts/data_setup.py
!git commit -m "add create dataLoaders function in data_setup.py script"

[add_data_setup_script 6cb216c] add create dataLoaders function in data_setup.py script
 1 file changed, 67 insertions(+)
 create mode 100644 scripts/data_setup.py


In [26]:
!git checkout main
!git merge add_data_setup_script
!git push origin main

M	notebooks/pytorch_going_modular_script_mode.ipynb
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 19fd07d..6cb216c
Fast-forward
 scripts/data_setup.py | 67 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 67 insertions(+)
 create mode 100644 scripts/data_setup.py
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 1010 bytes | 1010.00 KiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/guyfuchs/mini-food-pic-classifier
   19fd07d..6cb216c  main -> main


##3. Making a model (TinyVGG)
We're going to use the TinyVGG model from the CNN Explainer websitק, placing the it's class `TineVGG()` in a script called `model_builder.py`.




In [11]:
%%writefile scripts/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""

import torch

from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing scripts/model_builder.py


Now let's create an instance of `TinyVGG` (from a script) and put it on the target device.

>Note: If you're using Google Colab, and you'd like to use a GPU (recommended), you can turn one on via going to Runtime -> Change runtime type -> Hardware accelerator -> GPU.

In [22]:
import torch

from scripts import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model
torch.manual_seed(42)
model_0 = model_builder.TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                  hidden_units=10,
                  output_shape=len(class_names)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)


Let's check out our model by doing a dummy forward pass.

In [27]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))

# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0196, -0.0014,  0.0108]])

Output prediction probabilities:
tensor([[0.3366, 0.3297, 0.3337]])

Output prediction label:
tensor([0])

Actual label:
2


###Update Git

In [30]:
!cp {notebook_path} ./notebooks/
!git checkout -b create_model_instance_script

Switched to a new branch 'create_model_instance_script'


In [32]:
!git add notebooks/{notebook_name}
!git add scripts/model_builder.py
!git commit -m "instantiate dataloaders and model and test them"

[create_model_instance_script 56365c1] instantiate dataloaders and model and test them
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_script_mode.ipynb (97%)


In [33]:
!git checkout main
!git merge create_model_instance_script
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 6cb216c..56365c1
Fast-forward
 notebooks/pytorch_going_modular_script_mode.ipynb |  2 +-
 scripts/model_builder.py                          | 58 +++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 59 insertions(+), 1 deletion(-)
 create mode 100644 scripts/model_builder.py
Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 4.39 KiB | 2.19 MiB/s, done.
Total 8 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   6cb216c..56365c1  main -> main


##4. Pack `train_step()` and `test_step()` functions and `train()` into `engine.py` script
Let's place `train_step()`, `test_step()` and `train()` in the `engine.py` module.

In [35]:
%%writefile scripts/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
from typing import Dict, List, Tuple

import torch

from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = { "train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

Writing scripts/engine.py


###Update Git

In [36]:
!cp {notebook_path} ./notebooks/
!git checkout -b pack_train_test_functions_in_engine_script

Switched to a new branch 'pack_train_test_functions_in_engine_script'


In [37]:
!git add notebooks/{notebook_name}
!git add scripts/engine.py
!git commit -m "put train and steps functions in engine.py"

[pack_train_test_functions_in_engine_script c034810] put train and steps functions in engine.py
 2 files changed, 194 insertions(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_script_mode.ipynb (97%)
 create mode 100644 scripts/engine.py


In [38]:
!git checkout main
!git merge pack_train_test_functions_in_engine_script
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 56365c1..c034810
Fast-forward
 notebooks/pytorch_going_modular_script_mode.ipynb |   2 +-
 scripts/engine.py                                 | 193 ++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 194 insertions(+), 1 deletion(-)
 create mode 100644 scripts/engine.py
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 4.52 KiB | 2.26 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   56365c1..c034810  main -> main


##5. Add a function to save the model to utils.py
Let's setup a `utils.py` module to contain various utility functions, and put there the function to save our model to a directory.

In [39]:
%%writefile scripts/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
from pathlib import Path

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing scripts/utils.py


###Update Git

In [43]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_save_model_func_to_utils_script

Switched to a new branch 'add_save_model_func_to_utils_script'


In [44]:
!git add notebooks/{notebook_name}
!git add scripts/utils.py
!git commit -m "add utils module and a function to save the model"

[add_save_model_func_to_utils_script 68f1566] add utils module and a function to save the model
 2 files changed, 37 insertions(+), 1 deletion(-)
 create mode 100644 scripts/utils.py


In [45]:
!git checkout main
!git merge add_save_model_func_to_utils_script
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating a0505fd..68f1566
Fast-forward
 notebooks/pytorch_going_modular_script_mode.ipynb |  2 +-
 scripts/utils.py                                  | 36 ++++++++++++++++++++++++++++++++++++
 2 files changed, 37 insertions(+), 1 deletion(-)
 create mode 100644 scripts/utils.py
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.48 KiB | 1.48 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   a0505fd..68f1566  main -> main


##6. Train, evaluate and save the model
Let's combine all of our modular files into a single script `train.py`.

This will allow us to run all of the functions we've written with a single line of code on the command line:
```
python going_modular/train.py
```
Or if we're running it in a notebook:
```
!python going_modular/train.py
```
We'll go through the following steps:

1. Import the various dependencies, namely `torch`, `os`, `torchvision`.transforms and all of the scripts from the `scripts` directory, `data_setup`, `engine`, `model_builder`, `utils`.
* **Note:** Since `train.py` will be inside the going_modular directory, we can import the other modules via `import ...` rather than `from going_modular import ...`.
2. Setup various hyperparameters such as batch size, number of epochs, learning rate and number of hidden units (these could be set in the future via Python's `argparse`).
3. Setup the training and test directories.
4. Setup device-agnostic code.
5. Create the necessary data transforms.
6. Create the DataLoaders using `data_setup.py`.
7. Create the model using `model_builder.py`.
8. Setup the loss function and optimizer.
9. Train the model using `engine.py`.
10. Save the model using `utils.py`.

In [14]:
%%writefile scripts/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os

import torch

from torchvision import transforms

import get_data, data_setup, engine, model_builder, utils

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Project path is the parent of this script folder
project_path = os.path.dirname(os.path.abspath(__file__))

# Download data and get it paths
train_dir, test_dir = get_data.download_data(
                        project_path = project_path,
                        data_subpath = "pizza_steak_sushi",
                        file_name = "pizza_steak_sushi.zip",
                        source_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                        data_sesc = "pizza, steak, sushi data"
                    )
# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="going_modular_script_mode_tinyvgg_model.pth")

Overwriting scripts/train.py


###Update Git

In [47]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_train_script

Switched to a new branch 'add_train_script'


In [48]:
!git add notebooks/{notebook_name}
!git add scripts/train.py
!git commit -m "add train.py script"

[add_train_script d798220] add train.py script
 1 file changed, 68 insertions(+)
 create mode 100644 scripts/train.py


In [49]:
!git checkout main
!git merge add_train_script
!git push origin main

M	notebooks/pytorch_going_modular_script_mode.ipynb
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 68f1566..d798220
Fast-forward
 scripts/train.py | 68 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 68 insertions(+)
 create mode 100644 scripts/train.py
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.04 KiB | 1.04 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   68f1566..d798220  main -> main


Now, that all the parts are in scripts, let's run `train.py` file from the command line with:
```
!python scripts/train.py
```

In [52]:
!python scripts/train.py

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1063 | train_acc: 0.3047 | test_loss: 1.0983 | test_acc: 0.3011
 20% 1/5 [00:01<00:07,  1.77s/it]Epoch: 2 | train_loss: 1.0998 | train_acc: 0.3281 | test_loss: 1.0697 | test_acc: 0.5417
 40% 2/5 [00:03<00:04,  1.67s/it]Epoch: 3 | train_loss: 1.0869 | train_acc: 0.4883 | test_loss: 1.0808 | test_acc: 0.4924
 60% 3/5 [00:05<00:03,  1.67s/it]Epoch: 4 | train_loss: 1.0842 | train_acc: 0.3984 | test_loss: 1.0608 | test_acc: 0.5833
 80% 4/5 [00:06<00:01,  1.62s/it]Epoch: 5 | train_loss: 1.0662 | train_acc: 0.4141 | test_loss: 1.0654 | test_acc: 0.5644
100% 5/5 [00:08<00:00,  1.62s/it]
[INFO] Saving model to: models/going_modular_script_mode_tinyvgg_model.pth


###Update Git

In [55]:
!cp {notebook_path} ./notebooks/
!git checkout -b run_train_evaluate_save_model_via_script

fatal: A branch named 'run_train_evaluate_save_model_via_script' already exists.


In [56]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git add scripts/train.py
!git commit -m "run train, evaluate and save the model via script, fix minor error"

[run_train_evaluate_save_model_via_script d48a4f2] run train, evaluate and save the model via script, fix minor error
 1 file changed, 1 insertion(+), 1 deletion(-)


In [57]:
!git checkout main
!git merge run_train_evaluate_save_model_via_script
!git push origin main

M	notebooks/pytorch_going_modular_script_mode.ipynb
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating d798220..d48a4f2
Fast-forward
 scripts/train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 390 bytes | 390.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   d798220..d48a4f2  main -> main


##Appendix A: Create Dependencies

In [13]:
%%writefile requirements.txt
requests
torch
torchvision
tqdm

Writing requirements.txt


###Update Git

In [17]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_requirements_file_and_update_notebook

Switched to a new branch 'add_requirements_file_and_update_notebook'


In [18]:
!git add notebooks/{notebook_name}
!git add requirements.txt
!git commit -m "add requirements.txt file and update notebook"

[add_requirements_file_and_update_notebook 194be94] add requirements.txt file and update notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_script_mode.ipynb (96%)


In [19]:
!git checkout main
!git merge add_requirements_file_and_update_notebook
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating dca0024..194be94
Fast-forward
 notebooks/pytorch_going_modular_script_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.74 KiB | 1.87 MiB/s, done.
Total 4 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   dca0024..194be94  main -> main
